In [7]:
%reload_ext autoreload
%autoreload 2

current_jupyter_path: str = %pwd # type: ignore
while len(current_jupyter_path) > 1 and not current_jupyter_path.endswith("finance-tracker-agent"):
    %cd ..
    current_jupyter_path = %pwd # type: ignore

In [8]:
from src.openai_module import generate_text_with_conversation
from src.prompts import react_system_prompt
from src.json_helpers import extract_json

In [9]:
def save_expense(year, month, main_category, sub_category, account, amount, note):
    # This function would save the expense to a database or file.
    print(
        f"\n\n Expense saved: {year}, {month}, {main_category}, {sub_category}, {account}, {amount}, {note}"
    )


available_tools = {"save_expense": save_expense}

In [10]:
prompt = "I would like to save an expense: new tshirt for 15€"
prompt = "Last night I went for some drinks with friends and spent 20€"
prompt = "Spent 50€ on groceries 17 June 2025"
prompt = "Brought cigs for 10€ on 15 July 2025"
messages = [
    {"role": "system", "content": react_system_prompt},
    {"role": "user", "content": prompt},
]

In [11]:
turn_count = 1
max_turns = 3

while turn_count < max_turns:

    response = generate_text_with_conversation(messages=messages)

    print("-------------------------------")
    print(f"Turn {turn_count}:")
    print("Response:", response)

    # Extract tool calls from the response

    json_action = extract_json(response)

    if json_action:
        function_name = json_action[0].get("function_name")
        function_params = json_action[0]["function_parms"]

        print(f"Function Name: {function_name}")
        print(f"Function Params: {function_params}")

        if function_name in available_tools:

            tool = available_tools[function_name]
            tool_response = tool(**function_params)

            # print(f"Tool Response: {tool_response}")

            messages.append(
                {"role": "user", "content": f"Response from the tool: {tool_response}"}
            )
            break
    else:
        print("\n\n No tool calls found in the response.")
        break
    turn_count += 1

-------------------------------
Turn 1:
Response: Thought: The expense is for cigarettes, which falls under the 'Drinks' category. The sub-category would be 'Drinks'. The account used is likely the 'Main Account'. The year is 2025, and the month is July. The amount spent is 10.0€, and no specific note is given.

Action: 

{
  "function_name": "save_expense",
  "function_parms": {
    "year": 2025, 
    "month": 7, 
    "main_category": "Drinks", 
    "sub_category": "Drinks", 
    "account": "Main Account", 
    "amount": 10.0, 
    "note": ""
  }
}

PAUSE
Function Name: save_expense
Function Params: {'year': 2025, 'month': 7, 'main_category': 'Drinks', 'sub_category': 'Drinks', 'account': 'Main Account', 'amount': 10.0, 'note': ''}


 Expense saved: 2025, 7, Drinks, Drinks, Main Account, 10.0, 


In [7]:
json_action